USe Models trained on optimized data as its performance is very good videos as change in image i=and face width and height

In [1]:
%%capture
!pip install mediapipe

In [2]:
import matplotlib.pyplot as plt
import cv2
import pickle
from math import cos, sin, atan2
import mediapipe
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow

In [3]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):
    # print(pitch,yaw,roll)
    # cv2_imshow(img)
    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [11]:
# Extract face features and return data frame containing these features and nose points to draw data from
def extract_face_landmark(face_landmark):
    data = dict()
    for idx, lm in enumerate(face_landmark):
        # features number 10 and 152 represents the most botton and top features in the face
        dist = np.sqrt((face_landmark[152].x-face_landmark[10].x)**2 + (face_landmark[152].y-face_landmark[10].y)**2)

        # feature number 1 represents nose point
        dx = (lm.x - face_landmark[1].x) / dist
        dy = (lm.y - face_landmark[1].y) / dist
        data.setdefault(str(idx)+'_x',[])
        data[str(idx)+'_x'].append(dx)
        data.setdefault(str(idx)+'_y',[])
        data[str(idx)+'_y'].append(dy)
    return pd.DataFrame(data)

In [12]:
def nose_point(face_landmark, img_h, img_w):
    # feature number 1 represents nose point
    nose_2d = (face_landmark[1].x * img_w, face_landmark[1].y * img_h)
    return int(nose_2d[0]), int(nose_2d[1])

In [13]:
def predict(data,model_path):
    model = pickle.load(open(model_path, 'rb'))
    y_pred = model.predict(pd.DataFrame(data))

    return  y_pred[0][0], y_pred[0][1], y_pred[0][2]

In [15]:
cap = cv2.VideoCapture('/content/Pose_estimate video.mp4')
i = 0
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


fourcc = cv2.VideoWriter_fourcc('M','J','P','G')   
out = cv2.VideoWriter('/content/draw_pose_final.avi', fourcc, fps, (frame_width, frame_height))
faceModule = mediapipe.solutions.face_mesh
with faceModule.FaceMesh(static_image_mode=True) as faces:
  while cap.isOpened():
      ret, image = cap.read()

      if not ret:
        break
      # Flip the image horizontally for a later selfie-view display
      # Also convert the color space from BGR to RGB
      image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
      # To improve performance
      image.flags.writeable = False
      # Get the result
      results = faces.process(image)
      # To improve performance
      image.flags.writeable = True
      # Convert the color space from RGB to BGR
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      # for landmark in face.landmark:
      img_h, img_w, img_c = image.shape
      face_3d = []
      face_2d = []
      if results.multi_face_landmarks != None:
          for face in results.multi_face_landmarks:
              # extract face features
              data_df = extract_face_landmark(face.landmark)
              # extract x and y coordinates of nose point
              nose_x, nose_y = nose_point(face.landmark, img_h, img_w)
              # predict pitch yaw roll of the face from x y features
              pitch, yaw, roll = predict(data_df,'/content/Model.sav')
              # draw pitch yaw roll on the face
              image = draw_axis(image,pitch,yaw,roll,nose_x,nose_y)
          
      out.write(image)

      if cv2.waitKey(20) == ord('q'):
          break

      i+=1

  cv2.destroyAllWindows()
  cap.release()
  # print('done')